1. Plan travel using resrobot route planner
  - a) Find the stops names between Göteborg and Lund for a train that is departuring from Göteborg.

  - b) Find stops names, and times between two different cities of your choice in Sweden.

  - c) See if you can plot in a map points of a train on each stops between two locations of your choice.

2. Make digital display boards for trains/trams
  - a) Make a departure board for trams and buses for a station of your choice, e.g. Göteborg korsvägen. It displays how many minutes are left to wait. An example could be to print out

spårvagn 8 mot angered 2 minuter
spårvagn 2 mot högsbohöjd 5 minuter
...
  - b) Make a departure board for trains. Here list the trains, their destinations and the time it departures.

  - c) Make an arrival board for trains. Here list the trains, their destinations and the time it arrives.

In [19]:
from dotenv import load_dotenv
import os 
import requests


load_dotenv() # läser in miljövariablen från .env filen 

API_KEY = os.getenv("API_KEY") # hämtar värdet från nyckeln 



url = f"https://api.resrobot.se/v2.1/departureBoard?id=740000002&format=json&accessId={API_KEY}" # id = göteborgs centralstation, formatet är json och min api nyckel

response = requests.get(url)

# print(response) -- Detta gav svar <Response [200]> vilket menas att api anropet lyckades.

data = response.json() # detta konventerar datan från json till python objektformat i detta fall en dict.

# Extrahera alla avgångar
departures = data.get("Departure", [])[:5] # Hämtar listan med avgångar (5 första!)

# Iterera genom avgångarna och skriv ut relevanta detaljer
for departure in departures:
    line_name = departure.get("ProductAtStop", {}).get("name")  # Namn på linjen
    operator = departure.get("ProductAtStop", {}).get("operator")  # Operatör
    time = departure.get("time")  # Avgångstid
    print(f"Linje: {line_name}, Operatör: {operator}, Avgångstid: {time}")




Linje: Länstrafik - Buss 17, Operatör: Västtrafik, Avgångstid: 13:46:00
Linje: Länstrafik - Spårväg 6, Operatör: Västtrafik, Avgångstid: 13:46:00
Linje: Länstrafik - Buss X4, Operatör: Västtrafik, Avgångstid: 13:47:00
Linje: Länstrafik - Buss X4, Operatör: Västtrafik, Avgångstid: 13:48:00
Linje: Länstrafik - Buss 16, Operatör: Västtrafik, Avgångstid: 13:48:00


In [52]:
#  a) Find the stops names between Göteborg and Lund for a train that is departuring from Göteborg.

#stop lookup api anrop
url = f"https://api.resrobot.se/v2.1/location.name?input=stockholm&format=json&accessId={API_KEY}" # hämtar här Stop eller extID för stationerna. just nu inställd på göteborg vilket ger extId': '740098001' Lund gav 740098038

response = requests.get(url)

lund = response.json()

lund

{'stopLocationOrCoordLocation': [{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'},
      'cls': '2'},
     {'icon': {'res': 'prod_gen'}, 'cls': '4'},
     {'icon': {'res': 'prod_gen'}, 'cls': '8'},
     {'icon': {'res': 'prod_gen'}, 'cls': '16'},
     {'icon': {'res': 'prod_gen'}, 'cls': '32'},
     {'icon': {'res': 'prod_gen'}, 'cls': '64'},
     {'icon': {'res': 'prod_gen'}, 'cls': '128'}],
    'timezoneOffset': 60,
    'id': 'A=1@O=STOCKHOLM@X=18058151@Y=59330136@U=1@L=740098000@B=1@p=1736742989@',
    'extId': '740098000',
    'name': 'STOCKHOLM',
    'lon': 18.058151,
    'lat': 59.330136,
    'weight': 32767,
    'products': 254,
    'meta': True,
    'minimumChangeDuration': 'PT26M'}},
  {'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'},
      'cls': '2'},
     {'icon': {'res': 'prod_gen'}, 'cls': '4'},
     {'icon': {'res': 'prod_gen'}, 'cls': '8'},
     {'icon': {'res': 'prod_gen'}, 'cls': '16'},
     {'icon': {'res': 'prod_gen'}, 'cls': '32'},
 

In [46]:
#  a) Find the stops names between Göteborg and Lund for a train that is departuring from Göteborg.

# Routeplanner API-anrop
url = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000002&destId=740098038&passlist=1&showPassingPoints=true&accessId={API_KEY}"  # passlist = 1 inkluderar en lista med alla passerade stopp.

response = requests.get(url)  # Gör API-anropet.

stops = response.json()  # Konverterar svaret från JSON till en Python-dictionary.

trip = stops.get("Trip", [])[0]  # Hämtar den första resan i resultatet från anropet.

pass_list = trip.get("LegList", {}).get("Leg", [])  # Hämtar alla delar (legs) av resan.

for leg in pass_list:
    stoppen = leg.get("Stops", {}).get("Stop", [])  # Hämtar alla stopp för detta ben av resan.
    for stop in stoppen:
        stop_name = stop.get("name")  # Hämtar namnet på stoppet.
        times = stop.get("arrTime")
        print(f"Stop: {stop_name} Time: {times}")  


Stop: Göteborg Centralstation Time: None
Stop: Mölndal station Time: 14:48:00
Stop: Kungsbacka station Time: 14:59:00
Stop: Varberg station Time: 15:25:00
Stop: Falkenberg station Time: 15:45:00
Stop: Halmstad Centralstation Time: 16:03:00
Stop: Laholm station Time: 16:25:00
Stop: Båstad station Time: 16:32:00
Stop: Ängelholm station Time: 16:43:00
Stop: Helsingborg Centralstation Time: 17:04:00
Stop: Landskrona station Time: 17:19:00
Stop: Lund Centralstation Time: 17:36:00


In [58]:
#  b) Find stops names, and times between two different cities of your choice in Sweden.

extid_1 = 740000001 # sthlm
extid_2 = 740000003 # malmö

def route_planner (dep_id, arr_id):
    url = f"https://api.resrobot.se/v2.1/trip?format=json&originId={dep_id}&destId={arr_id}&passlist=1&showPassingPoints=true&accessId={API_KEY}"

    response = requests.get(url)  # Gör API-anropet.

    stops = response.json()  # Konverterar svaret från JSON till en Python-dictionary.

    trip = stops.get("Trip", [])[0]  # Hämtar den första resan i resultatet från anropet.

    pass_list = trip.get("LegList", {}).get("Leg", [])  # Hämtar alla delar (legs) av resan.

    stops_and_times = [] # Lista för att lagra alla stopp och tider

    for leg in pass_list:
        stoppen = leg.get("Stops", {}).get("Stop", [])  # Hämtar alla stopp för detta ben av resan.
        for stop in stoppen:
            stop_name = stop.get("name")  # Hämtar namnet på stoppet.
            times = stop.get("arrTime")
            stops_and_times.append((stop_name, times))  # Lägg till tuple i listan
    
    
    return stops_and_times

test = route_planner(extid_1, extid_2)


for stop_name, times in test:  # Iterera genom varje tuple i listan
    print(f"Stopp: {stop_name}, Tid: {times}")




Stopp: Stockholm Centralstation, Tid: None
Stopp: Norrköping Centralstation, Tid: 16:39:00
Stopp: Linköping Centralstation, Tid: 17:05:00
Stopp: Nässjö Centralstation, Tid: 17:54:00
Stopp: Alvesta station, Tid: 18:27:00
Stopp: Älmhult station, Tid: 18:49:00
Stopp: Hässleholm Centralstation, Tid: 19:11:00
Stopp: Lund Centralstation, Tid: 19:40:00
Stopp: Malmö Centralstation, Tid: 19:52:00


In [ ]:
# c) See if you can plot in a map points of a train on each stops between two locations of your choice.

import folium 



